In [1]:
import json
import numpy as np
from data_loader import get_loader
import os
import torch

In [2]:
experiment = "base"

# base architecture
if "base" in experiment:
    from configs import base_config as cfg
else:
    experiment = "dummy"

# Get argument from config
cfg = cfg.cfg
num_workers = cfg["num_workers"]
epochs = cfg["epochs"]
model_type = experiment
data_path = "../preprocess/TR6614_no_pssm_onedim.json"
batch_size = 2

In [3]:
data = json.load(open("../preprocess/TR6614_no_pssm_onedim.json", "r"))
ids = np.arange(len(data))

In [4]:
experiment = "dummy"

In [5]:

if experiment == "dummy":
    loader, _ = get_loader(protein_data=data,
                           ids=[0, 1, 2, 3, 5],
                           batch_size=batch_size,
                           shuffle=False,
                           num_workers=num_workers)
else:
    loader, _ = get_loader(protein_data=data,
                           ids=ids,
                           batch_size=batch_size,
                           shuffle=False,
                           num_workers=num_workers)

latest_model_path = os.path.join("models", model_type, "latest_model.pt")

if os.path.exists(latest_model_path):
    print("Model exists. Loading from {0}".format(latest_model_path))
    if torch.cuda.is_available():
        model = torch.load(latest_model_path)
        device = "cuda"
    else:
        model = torch.load(latest_model_path, map_location=torch.device('cpu'))
        device = "cpu"
else:
    print("Model not trained")
    print("Exiting...")

# generated_sequences = gen(model, loader, device)

# with open(stats_path, "rb") as f:
#     stats_dict = pkl.load(f)

# stats_dict["test"]["acc"] = acc

# with open(stats_path, "wb") as f:
#     pkl.dump(stats_dict, f)

Loading 0/5 proteins
5 5 5
Model exists. Loading from models/base/latest_model.pt


In [6]:
def gen(model, loader, device):
    fmt_string = "Batch[{0}/{1}]"
    
    gen_sequences = []
    model.eval()
    num_batches = len(loader)
    
    for iter, (X, _, _) in enumerate(loader):
        batch_loss = 0

        X = X.permute([0, 2, 1]).long().to(device)
        outputs = model.gen(X)

        print(fmt_string.format(iter, num_batches))
        gen_sequences.append(outputs.detach().cpu().numpy())


    return gen_sequences


In [7]:
PSSM = gen(model, loader, device)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Batch[0/3]
Batch[1/3]
Batch[2/3]


In [8]:
PSSM = np.vstack(PSSM)

In [10]:
PSSM.shape

(5, 700, 21)

In [11]:
one_dim_conservation = np.log(21) + np.sum(PSSM * np.log(PSSM + 1e-4), axis=2, keepdims=True)

In [17]:
PSSM = np.concatenate([PSSM, one_dim_conservation], axis=2)

In [18]:
PSSM.shape

(5, 700, 22)

In [12]:
if not os.path.exists("gen_seq"):
    os.mkdir("gen_seq")